[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LLaMA-VID-jupyter/blob/main/test.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaMA-VID
%cd /content/LLaMA-VID

# !pip install -q llava@git+https://github.com/haotian-liu/LLaVA.git@1619889c712e347be1cb4f78ec66e7cf414ac1a6
# !pip install -q einops fastapi gradio markdown2[all] uvicorn wandb shortuuid httpx deepspeed peft accelerate 
# !pip install -q bitsandbytes einops einops-exts timm gradio_client fairscale decord transformers==4.31.0
!pip install -q -e .
!pip install -q ninja
!pip install -q flash-attn --no-build-isolation

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://storage.googleapis.com/sfr-vision-language-research/LAVIS/models/BLIP2/eva_vit_g.pth -d /content/LLaMA-VID/model_zoo/LAVIS -o eva_vit_g.pth

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/tokenizer_config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/vocab.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o vocab.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/special_tokens_map.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/raw/main/preprocessor_config.json -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o preprocessor_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -d /content/LLaMA-VID/model_zoo/OpenAI/clip-vit-large-patch14 -o pytorch_model.bin

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/config.json -d /content/vid -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/generation_config.json -d /content/vid -o generation_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00001-of-00003.bin -d /content/vid -o pytorch_model-00001-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00002-of-00003.bin -d /content/vid -o pytorch_model-00002-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/pytorch_model-00003-of-00003.bin -d /content/vid -o pytorch_model-00003-of-00003.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/pytorch_model.bin.index.json -d /content/vid -o pytorch_model.bin.index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/special_tokens_map.json -d /content/vid -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/tokenizer.json -d /content/vid -o tokenizer.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/resolve/main/tokenizer.model -d /content/vid -o tokenizer.model
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/4bit/llama-vid-7b-full-224-long-video-5GB/raw/main/tokenizer_config.json -d /content/vid -o tokenizer_config.json

In [ ]:
%cd /content/LLaMA-VID

import torch

from llamavid.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llamavid.conversation import conv_templates, SeparatorStyle
from llamavid.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image
from decord import VideoReader, cpu

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer

def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

def load_video(video_path, fps=1):
    vr = VideoReader(video_path, ctx=cpu(0))
    fps = round(vr.get_avg_fps()/fps)
    frame_idx = [i for i in range(0, len(vr), fps)]
    spare_frames = vr.get_batch(frame_idx).asnumpy()
    return spare_frames

disable_torch_init()

# from llamavid.model import LlavaLlamaAttForCausalLM
# model = LlavaLlamaAttForCausalLM.from_pretrained('/content/vid', load_in_8bit=True)
model_name = get_model_name_from_path('/content/vid')
tokenizer, model, image_processor, context_len = load_pretrained_model('/content/vid', None, model_name, load_8bit=True, load_4bit=False, device_map="cuda", device="cuda")
conv_mode = "llava_v1"
conv = conv_templates[conv_mode].copy()
roles = conv.roles

In [ ]:
image = load_image('/content/titanic.jpg')
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()
# inp = input(f"{roles[0]}: ")
inp = 'explain'
model.update_prompt([[inp]])
inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
conv.append_message(conv.roles[0], inp)
image = None
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
prompt

input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
with torch.inference_mode():
  output_ids = model.generate(
      input_ids,
      images=image_tensor,
      do_sample=True,
      temperature=0.2,
      top_p=0.7,
      max_new_tokens=512,
      streamer=streamer,
      use_cache=True,
      stopping_criteria=[stopping_criteria])

In [ ]:
outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
conv.messages[-1][-1] = outputs
conv.messages[-2][-1] = conv.messages[-2][-1].replace(DEFAULT_IMAGE_TOKEN+'\n','')
print("\n", {"prompt": prompt, "outputs": outputs}, "\n")